In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks/hw3

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/hw3


# **Task 1:  Sequence Tagging with RNNs**

In this task, you will implement LSTM and Bi-LSTM architectures with PyTorch to perform part-of-speech tagging (a sequence tagging task).

### **Data**
We use a subset of the data from the CoNLL-2003 shared task on Named Entity Recognition (provided in the zip). It is pre-partioned into a training, development and test set.

The dataset consists of pre-tokenized sentences where every token is annotated with a part-of-speech tag, a syntactic chunk tag and a named entity tag. In this home exercise, we only use the IOB named entity recognition tag.

In [2]:
import torch
import torch.nn as nn
import os
import re
import numpy as np
from collections import defaultdict
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt

## **Task 1.1: Pretrained Embeddings (5p)**

Download the pretrained, uncased GloVe embeddings with 6B tokens [glove.6B.zip](https://nlp.stanford.edu/projects/glove/) from Stanford.

For performance reasons, we will only use the 50-dimensional embeddings **glove.6B.50d.txt**.

Implement a function to read the embedding and another function to read the dataset.

In [32]:
# https://medium.com/analytics-vidhya/ner-tensorflow-2-2-0-9f10dcf5a0a
"""
  Store each sentence seperately. Each token of the sentences is stored with its corresponding IOB named entity recognition tag
"""

def split_text_label(filename):
  f = open(filename)
  split_labeled_text = []
  sentence = []
  for line in f:
    if len(line)==0 or line.startswith('-DOCSTART') or line[0]=="\n":
       if len(sentence) > 0:
         split_labeled_text.append(sentence)
         sentence = []
       continue
    splits = line.split(' ')
    sentence.append([splits[0],splits[-1].rstrip("\n")])
  if len(sentence) > 0:
    split_labeled_text.append(sentence)
    sentence = []
  return split_labeled_text

In [36]:
def read_data(path):
    data = defaultdict(list)
    label = defaultdict(list)
    # TODO: YOUR CODE HERE

    text_label = split_text_label(filename=path)
    for idx, sent in enumerate(text_label):
      words = []
      iob_labels = []
      for word_iob in sent:
        words.append(word_iob[0]) #add word
        iob_labels.append(word_iob[1]) #add iob tag

      data[idx] = words
      label[idx] = iob_labels

    return data, label

def get_pretrained_embeddings(embedding_path):
    embeddings = defaultdict(list)
    # TODO: YOUR CODE HERE
    with open("glove.6B.50d.txt", "r") as file:
      for line in file:
        line = line.strip()
        word = line.split(" ")[0]
        vector = np.array([float(n) for n in line.split(" ")[1:]])

        embeddings[word] = vector

    return embeddings

In [4]:
embed = get_pretrained_embeddings(embedding_path="glove.6B.50d.txt")

In [10]:
embed["get"].shape

(50,)

In [25]:
out= split_text_label("data/ner_eng_bio.train")

In [35]:
out[2]

[['BRUSSELS', 'B-LOC'], ['1996-08-22', 'O']]

In [37]:
data, label = read_data("data/ner_eng_bio.train")

In [52]:
data[0]

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [53]:
label[0]

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [11]:
counter = 0
with open("data/ner_eng_bio.train", "r") as file:
    for line in file:
      print(line.strip().split(" "))
      if counter == 10:
        break
      counter += 1

['EU', 'NNP', 'B-NP', 'B-ORG']
['rejects', 'VBZ', 'B-VP', 'O']
['German', 'JJ', 'B-NP', 'B-MISC']
['call', 'NN', 'I-NP', 'O']
['to', 'TO', 'B-VP', 'O']
['boycott', 'VB', 'I-VP', 'O']
['British', 'JJ', 'B-NP', 'B-MISC']
['lamb', 'NN', 'I-NP', 'O']
['.', '.', 'O', 'O']
['']
['Peter', 'NNP', 'B-NP', 'B-PER']


In [ ]:
"""dataset = defaultdict(list)
labels = defaultdict(list)
with open("data/ner_eng_bio.train", "r") as file:
    sentence = []
    sentence_labels = []
    for line in file:
        line = line.strip()
        if line == "":
            if sentence:
                for word, label in zip(sentence, sentence_labels):
                    dataset["words"].append(word)
                    labels["tags"].append(label)
            sentence = []
            sentence_labels = []
        else:
            word, _, _, label = line.split()
            sentence.append(word)
            sentence_labels.append(label)"""

## **Task 1.2: LSTM and Bi-LSTM Model (10p)**

We will use PyTorch to build our LSTM. Complete the `__init__()` and the `forward()` function of the CustomLSTM class. The model will have the following components:
- A single [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) layer which takes the embeddings as input and has 100-dimensional hidden layer. The LSTM is **not** bidirectional.
- A dropout layer with probability 0.1
- A linear layer with input size of 100 (the hidden layer size of the LSTM layer) and output size of the number of labels
- A [Sigmoid](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html) activation function

In [54]:
from torch.nn.modules import dropout
class CustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_labels):
        super(CustomLSTM, self).__init__()
        # TODO: YOUR CODE HERE
        """self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_labels = num_labels"""

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, dropout=0.1, bidirectional=False)
        self.linear = nn.Linear(input_size=100, out_features=num_labels)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # TODO: YOUR CODE HERE
        #LSTM’s output corresponding to all timesteps
        output, (h_n, c_n) = self.lstm(x)
        last_hidden_state = output[-1] # can also be h_n   see https://towardsdatascience.com/implementation-differences-in-lstm-layers-tensorflow-vs-pytorch-77a31d742f74#:~:text=The%20output%20of%20the%20Pytorch,another%20tuple%20with%20two%20elements.
        linear_output = self.linear(last_hidden_state)
        output = self.sigmoid(linear_output)
        
        return output

## **Task 1.3: Training Model (10p)**

Complete the function `train` to train your model. The model will train with batch size of 1 (each sentence split by "\n" is a sample) for 10 epochs. You will train the model with the train dataset and use dev dataset to check the model's performance after each epoch. Calculate the macro f1 score of the model on the dev set. Return the losses and f1 scores for plotting.

**Hint**: you can check out this [link](https://pytorch.org/tutorials/beginner/introyt/trainingyt.html#the-training-loop) to get to know more about how to train model with pytorch. For the f1 score you can use `sklearn.metrics.f1_score`, remember to set the `average` parameter to "macro".


In [55]:
def label_encode(labels):
    labels_list = {'B-LOC': 0, 'B-MISC': 1, 'B-ORG': 2, 'B-PER': 3, 'I-LOC': 4, 'I-MISC': 5, 'I-ORG': 6, 'I-PER': 7,
                   'O': 8}
    labels_encode = [labels_list[label] for label in labels]
    return labels_encode

In [ ]:
# Change this to your path to the dataset
train = "data/ner_eng_bio.train" 
test = "data/ner_eng_bio.test"
dev = "data/ner_eng_bio.dev"

embeddings = get_pretrained_embeddings("embeddings/glove.6B.50d.txt")

train_data, train_label = read_data(train)
test_data, test_label = read_data(test)
dev_data, dev_label = read_data(dev)

# Change the Hyperparameters here
input_size = 0
hidden_size = 0
num_layers = 0
num_labels = 0
epochs = 0

In [ ]:
model = CustomLSTM(input_size, hidden_size, num_layers, num_labels)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters()) 

def train(model, loss_fn, optimizer, train_data, dev_data, epochs):
    logs_loss = []
    logs_f1_score = []

    # Training
    for epoch in range(epochs):
        # Train with train set
        model.train()
        # TODO: YOUR CODE HERE

        # Evaluate with dev set
        with torch.no_grad():
            model.eval()
            # TODO: YOUR CODE HERE
        
    return logs_loss, logs_f1_score

logs_loss, logs_f1_score = train(model, loss_fn, optimizer, train_data, dev_data, epochs)

## **Task 1.4: Visualizing (5p)**

Check the performance of the model on the test set and plot the training loss using `matplotlib.pyplot.plot`.

In [ ]:
# Evaluate with test set
with torch.no_grad():
    model.eval()
    preds = []
    targets = []
    # TODO: YOUR CODE HERE

# Plot with matplotlib
